In [1]:
from PIL import Image
import torch
import os
import cv2
from tqdm import tqdm
import copy
import torch.optim as optim
import numpy as np
import optuna
from sklearn.metrics import f1_score
from optuna.exceptions import TrialPruned
import random
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from skimage import io

In [2]:
working_folder = os.path.abspath("")
image_dir = os.path.join(working_folder, "/kaggle/input/all-images/data_all")

In [3]:
train_df = '/kaggle/input/train-full/train_full.csv'

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 1]
        img_name = os.path.join(self.root_dir, f"{img_id}")
        image = Image.open(img_name).convert('RGB')
        label = int(self.annotations.iloc[index, 0][-1])  # Extract class number

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
class SingleFolderDataset(Dataset):
    def __init__(self, directory, transform):
        self.directory = directory
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.directory, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name

In [7]:
def make_predictions_sigmoid(data_loader):
    model.eval()
    y_pred = []
    file_names = []
    with torch.no_grad():
        for inputs, paths in (data_loader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            y_pred.extend(probabilities.cpu().numpy())
            file_names.extend([os.path.basename(path) for path in paths])
            
    return file_names, y_pred

In [8]:
for trail in range(0, 10):
    train_dataset = CustomImageDataset(train_df, image_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
    
    model = models.efficientnet_v2_l(weights = models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)
    
    cutoff_reached = False
    for name, param in model.named_parameters():
        if not cutoff_reached:
            if name == "features.2.6.block.1.1.bias":
                cutoff_reached = True
            param.requires_grad = False
        else:
            param.requires_grad = True

    num_ftrs = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, 1344),
        nn.ReLU(),
        nn.Dropout(0.55),
        nn.Linear(1344, 10),
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    print("model load_compelete")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=2.4017967647941986e-05)
    
    dataset = SingleFolderDataset(directory='/kaggle/input/state-farm-distracted-driver-detection/imgs/test', transform=transform)
    test_loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=4)

    def train_model(model, criterion, optimizer, num_epochs, trail):
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for inputs, labels in (train_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

            file_names, preds = make_predictions_sigmoid(test_loader)
            submission_df = pd.DataFrame(preds, columns=[f'c{i}' for i in range(10)])
            submission_df.insert(0, 'img', file_names)
            submission = "submission" + str(trail) + str(epoch) + ".csv"
            submission_df.to_csv(submission, index=False)       

        print('Finished Training')
    
    train_model(model, criterion, optimizer, 1, trail)
    

Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth
100%|██████████| 455M/455M [00:02<00:00, 180MB/s]


model load_compelete
Epoch 1, Loss: 0.3838671591236175
Finished Training
model load_compelete
Epoch 1, Loss: 0.4076405740658568
Finished Training
model load_compelete
Epoch 1, Loss: 0.39700911997019656
Finished Training
model load_compelete
Epoch 1, Loss: 0.39119720355389687
Finished Training
model load_compelete
Epoch 1, Loss: 0.40651260113898935
Finished Training
model load_compelete
Epoch 1, Loss: 0.402847204379556
Finished Training
model load_compelete
Epoch 1, Loss: 0.40371421216449005
Finished Training
model load_compelete
Epoch 1, Loss: 0.40008888252679997
Finished Training
model load_compelete
Epoch 1, Loss: 0.39285998248074006
Finished Training
model load_compelete
Epoch 1, Loss: 0.3986159513761607
Finished Training
